In [22]:
import keras
from keras.models import Sequential, Model
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.layers import Dense
import pandas as pd
batch_size = 32

# Data Pull
Data comes from ![Kaggle](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data)
My file structure is as follows: 
```
data/
    test/
        unk/
            12500 cat/dog unkown pictures
    train/
        cat/
            11250 cat pictures
        dog/
            11250 dog pictures
    valid/
        cat/
            1250 cat pictures
        dog/
            1250 dog pictures
sample/
    data/
        train/
            cat/
                1138 cat pictures
            dog/
                1138 dog pictures
        valid/
            cat/
                112 cat pictures
            dog/
                112 dog pictures
```

In [21]:
#path = "sample/data/"
path = "data/"

In [20]:
idg = image.ImageDataGenerator(rotation_range=0.1,horizontal_flip=True, channel_shift_range=0.2)
idg_test = image.ImageDataGenerator()
trn_batch = idg.flow_from_directory(path+"train/", target_size=(224,224), batch_size=batch_size, class_mode="binary")
val_batch = idg.flow_from_directory(path+"valid/", target_size=(224,224), batch_size=batch_size, class_mode="binary")
tst_batch = idg_test.flow_from_directory(path+"test/", target_size=(224,224), batch_size=batch_size)

Found 2276 images belonging to 2 classes.
Found 224 images belonging to 2 classes.


FileNotFoundError: [Errno 2] No such file or directory: 'sample/data/test/'

In [5]:
vgg16 = VGG16()
vgg16.layers.pop()
for layer in vgg16.layers:
    layer.trainable = False 
m = Dense(4096, activation="relu")(vgg16.layers[-1].output)
m = Dense(1, activation="sigmoid")(m)
vgg16 = Model(vgg16.input, m)
vgg16.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [6]:
vgg16.fit_generator(trn_batch, steps_per_epoch=(trn_batch.n/batch_size), validation_data=val_batch, validation_steps=(val_batch.n/batch_size), epochs = 5)

Epoch 1/5
704/703 [==============================] - 123s - loss: 0.7807 - acc: 0.9455 - val_loss: 0.8522 - val_acc: 0.9444
Epoch 2/5
704/703 [==============================] - 120s - loss: 0.6008 - acc: 0.9607 - val_loss: 0.5921 - val_acc: 0.9612
Epoch 3/5
704/703 [==============================] - 121s - loss: 0.8235 - acc: 0.9476 - val_loss: 0.6711 - val_acc: 0.9572
Epoch 4/5
704/703 [==============================] - 121s - loss: 0.6755 - acc: 0.9572 - val_loss: 0.7665 - val_acc: 0.9516
Epoch 5/5
704/703 [==============================] - 123s - loss: 0.7345 - acc: 0.9537 - val_loss: 1.1522 - val_acc: 0.9268


In [18]:
vgg16.fit_generator(trn_batch, steps_per_epoch=(trn_batch.n/batch_size), validation_data=val_batch, validation_steps=(val_batch.n/batch_size), epochs = 5)

Epoch 1/5
704/703 [==============================] - 122s - loss: 0.6804 - acc: 0.9577 - val_loss: 0.6864 - val_acc: 0.9572
Epoch 2/5
704/703 [==============================] - 120s - loss: 0.6775 - acc: 0.9577 - val_loss: 0.6479 - val_acc: 0.9592
Epoch 3/5
704/703 [==============================] - 122s - loss: 0.6790 - acc: 0.9574 - val_loss: 0.8370 - val_acc: 0.9472
Epoch 4/5
704/703 [==============================] - 119s - loss: 0.9714 - acc: 0.9390 - val_loss: 1.0341 - val_acc: 0.9352
Epoch 5/5
704/703 [==============================] - 116s - loss: 0.9208 - acc: 0.9426 - val_loss: 0.7850 - val_acc: 0.9512


In [7]:
predictions = vgg16.predict_generator(tst_batch, steps=(tst_batch.n/batch_size))

In [13]:
submission = pd.DataFrame(columns=['id','label'])

In [14]:
predictions[0][0]

0.0

In [15]:
for i in range(0,len(predictions)):
    pred_id = tst_batch.filenames[i].split("/")[1].split(".")[0]
    pred_label = predictions[i][0]
    if pred_label > 0.98: 
        pred_label = 0.98
    if pred_label < 0.02:
        pred_label = 0.02
    submission = submission.append({"id":pred_id, "label":pred_label}, ignore_index=True)

In [16]:
submission.to_csv("Submission_Dog_Cat_Redux_17_10_14_02.csv", index=False)